## 1. TF, IDF, TF-IDF 설명

**TF (Term Frequency, 단어 빈도):**

*   **정의:** 특정 단어가 문서 내에서 얼마나 자주 등장하는지를 나타내는 값입니다.
*   **계산:**
    ```
    TF(t, d) = 단어 t가 문서 d에 등장한 횟수
    ```
*   **특징:**
    *   TF 값이 높을수록 해당 단어가 문서에서 중요하다는 것을 의미할 수 있습니다.
    *   하지만, 불용어(stop words)처럼 자주 등장하지만 실제로는 중요하지 않은 단어들도 높은 TF 값을 가질 수 있습니다.

**IDF (Inverse Document Frequency, 역문서 빈도):**

*   **정의:** 특정 단어가 여러 문서에서 얼마나 드물게 등장하는지를 나타내는 값입니다.
*   **계산:**
    ```
    IDF(t, D) = log( N / (1 + DF(t)) )
    ```
    *   N: 전체 문서의 수
    *   DF(t): 단어 t가 등장한 문서의 수
    *   1을 더하는 이유: DF(t)가 0이 되는 경우 (즉, 어떤 단어가 모든 문서에 나타나지 않는 경우)를 방지하기 위함입니다.
    *   로그를 사용하는 이유: IDF 값의 범위를 조절하고, DF(t)의 작은 변화에 IDF 값이 과도하게 커지는 것을 방지하기 위해서 사용합니다.

*   **특징:**
    *   IDF 값이 높을수록 해당 단어가 특정 문서를 구별하는 데 중요한 역할을 한다는 것을 의미합니다.
    *   자주 등장하는 단어(예: 불용어)는 IDF 값이 낮아집니다.

**TF-IDF (Term Frequency - Inverse Document Frequency):**

*   **정의:** TF와 IDF 값을 곱하여 특정 단어의 중요도를 나타내는 값입니다.
*   **계산:**
    ```
    TF-IDF(t, d, D) = TF(t, d) * IDF(t, D)
    ```
*   **특징:**
    *   TF-IDF 값이 높으면 해당 단어가 특정 문서에서는 자주 등장하지만, 다른 문서에서는 드물게 등장한다는 것을 의미합니다. 즉, 해당 문서를 대표하는 중요한 단어일 가능성이 높습니다.
    *   TF와 IDF를 함께 사용하여 단어의 중요도를 보다 정확하게 평가할 수 있습니다.



## 2. DTM과 TF-IDF의 관계

문서 단어 행렬(DTM)은 TF-IDF를 계산하기 위한 기반 데이터로 사용됩니다.

1.  **DTM -> TF:** DTM의 각 셀(cell) 값은 해당 문서에서 해당 단어의 빈도(TF)를 나타냅니다.
2.  **DTM -> DF:** DTM의 열(column)을 보면, 각 단어가 몇 개의 문서에서 나타나는지 알 수 있습니다. 이것이 DF(Document Frequency)입니다.
3.  **TF & DF -> IDF:** DF를 이용하여 IDF를 계산합니다.
4.  **TF & IDF -> TF-IDF:** TF와 IDF를 곱하여 각 단어, 각 문서에 대한 TF-IDF 값을 계산합니다.

즉, DTM은 TF-IDF 계산에 필요한 모든 정보를 담고 있는 행렬입니다.



## 3. TfidfVectorizer를 사용하지 않고 TF-IDF 행렬 만들기 (Python)

```python
import numpy as np
from collections import defaultdict

def create_tfidf_matrix(documents):
    """
    문서 리스트를 입력받아 TF-IDF 행렬을 생성하는 함수.
    (TfidfVectorizer를 사용하지 않고 구현)

    Args:
        documents: 문서 리스트 (예: ["나는 영화가 좋다", "영화는 재미있다"])

    Returns:
        TF-IDF 행렬 (NumPy 배열), 단어 사전 (dict)
    """

    # 1. DTM 및 단어 사전 생성 (이전 코드 재사용)
    def create_dtm(documents):
      # 1. 단어 사전 만들기
      word_dict = defaultdict(lambda: len(word_dict))

      # 2. DTM 생성
      dtm = []
      for doc in documents:
          term_frequency = {}  # 문서 내 단어 빈도
          for word in doc.split():
              term_frequency[word_dict[word]] = term_frequency.get(word_dict[word], 0) + 1
          dtm.append(term_frequency) # dtm에 추가

      return dtm, word_dict
    
    dtm, word_dict = create_dtm(documents)

    # 2. TF 계산 (DTM을 NumPy 배열로 변환)
    num_docs = len(documents)
    num_words = len(word_dict)
    tf_matrix = np.zeros((num_docs, num_words))
    for i, doc_freq in enumerate(dtm):
        for word_id, freq in doc_freq.items():
            tf_matrix[i, word_id] = freq  # 문서 내 단어 빈도 (TF)

    # 3. DF 계산
    df = np.zeros(num_words)  # 각 단어별 문서 빈도
    for word_id in range(num_words):
      for doc_freq in dtm:
        if word_id in doc_freq:
          df[word_id] += 1


    # 4. IDF 계산
    idf = np.log(num_docs / (1 + df))

    # 5. TF-IDF 계산
    tfidf_matrix = tf_matrix * idf  # NumPy broadcasting 활용

    return tfidf_matrix, word_dict

# 예시 문서
documents = [
    "나는 영화가 좋다",
    "영화는 재미있다",
    "나는 영화가 싫다"
]

# TF-IDF 행렬 생성
tfidf_matrix, word_dict = create_tfidf_matrix(documents)
print("TF-IDF 행렬:\n", tfidf_matrix)
print("단어 사전:", word_dict)

```

**설명:**

1.  **`create_dtm` 함수:** 이전 답변에서 사용한 DTM 생성 함수를 재사용합니다.
2.  **TF 계산:**
    *   DTM을 NumPy 배열로 변환합니다. (`tf_matrix`)
    *   각 셀의 값은 해당 문서에서의 해당 단어의 빈도(TF)를 나타냅니다.
3.  **DF 계산:**
    *   `df`: 각 단어가 나타난 문서의 수를 저장하는 배열입니다.
4.  **IDF 계산:**
    *   `idf`: 각 단어에 대한 IDF 값을 계산합니다. NumPy의 로그 함수(`np.log`)를 사용합니다.
5.  **TF-IDF 계산:**
    *   `tfidf_matrix`: TF 행렬(`tf_matrix`)과 IDF 벡터(`idf`)를 곱하여 TF-IDF 행렬을 계산합니다. NumPy의 브로드캐스팅(broadcasting) 기능을 활용하여 효율적으로 계산합니다.



## 4. L2 정규화 (L2 Normalization)

**L2 정규화 (L2 Norm):**

*   **정의:** 벡터의 각 원소를 제곱하여 더한 후 제곱근을 취하는 연산입니다. 벡터의 크기(길이, magnitude)를 나타냅니다.
*   **계산 (벡터 x = [x1, x2, ..., xn]):**

    ```
    L2 Norm(x) = sqrt( x1^2 + x2^2 + ... + xn^2 )
    ```

*   **벡터 정규화:** 벡터를 L2 Norm으로 나누면, 벡터의 크기가 1이 됩니다. 즉, 방향은 유지하면서 크기만 1로 만듭니다.

    ```
    Normalized x = x / L2 Norm(x)
    ```

*   **TF-IDF에서의 L2 정규화:**  TF-IDF 행렬의 각 행(문서)에 대해 L2 정규화를 적용하면, 각 문서를 나타내는 벡터의 크기가 1이 됩니다. 이는 문서의 길이에 따른 영향을 줄이고, 코사인 유사도와 같은 거리 기반 유사도 측정 시 성능을 향상시키는 데 도움이 됩니다.

**Python 예시 (NumPy 사용):**

```python
import numpy as np

# TF-IDF 행렬 (위에서 계산한 tfidf_matrix 사용)
# 예시: tfidf_matrix = ...

# L2 Norm 계산
l2_norms = np.linalg.norm(tfidf_matrix, axis=1, keepdims=True)  # 각 행(문서)별 L2 Norm

# L2 정규화된 TF-IDF 행렬
normalized_tfidf_matrix = tfidf_matrix / l2_norms

print("L2 정규화된 TF-IDF 행렬:\n", normalized_tfidf_matrix)
```

**설명:**

*   `np.linalg.norm(tfidf_matrix, axis=1, keepdims=True)`:
    *   `axis=1`: 각 행(row)에 대해 L2 Norm을 계산합니다.
    *   `keepdims=True`: 결과를 (n, 1) 형태의 열 벡터로 유지합니다. (브로드캐스팅을 위해)
*   `tfidf_matrix / l2_norms`:  TF-IDF 행렬의 각 행을 해당 행의 L2 Norm으로 나눕니다. (L2 정규화)

L2 정규화는 텍스트 처리뿐만 아니라 머신 러닝의 다양한 분야에서 데이터 전처리 기법으로 널리 사용됩니다.
